In [3]:
from pyspark.sql.functions import min, max, to_timestamp, col
from pyspark.sql.functions import year, month, desc
from pyspark.sql import SparkSession

StatementMeta(, dab422a6-56ab-4b3e-9238-931bcf23aaf2, 5, Finished, Available)

## **Order Partition**

In [4]:
spark = SparkSession.builder \
    .appName("Read CSV with PySpark") \
    .getOrCreate()

# Read the CSV file into a DataFrame
df = spark.read.format("csv") \
           .option("header", "true") \
           .option("inferSchema", "true") \
           .load("Files/DataCoSupplyChainDataset.csv")

StatementMeta(, dab422a6-56ab-4b3e-9238-931bcf23aaf2, 6, Finished, Available)

In [5]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
df = df.withColumn("order date (DateOrders)", to_timestamp(df["order date (DateOrders)"], "M/d/yyyy HH:mm"))
df.select('order date (DateOrders)').printSchema()

StatementMeta(, dab422a6-56ab-4b3e-9238-931bcf23aaf2, 7, Finished, Available)

root
 |-- order date (DateOrders): timestamp (nullable = true)



In [6]:
result = df.select(min("order date (DateOrders)").alias("earliest_order_date"), max("order date (DateOrders)").alias("latest_order_date")).collect()

for row in result:
    print("Earliest Order Date:", row.earliest_order_date)
    print("Latest Order Date:", row.latest_order_date)

StatementMeta(, dab422a6-56ab-4b3e-9238-931bcf23aaf2, 8, Finished, Available)

Earliest Order Date: 2015-01-01 00:00:00
Latest Order Date: 2018-01-31 23:38:00


In [7]:
df = df.withColumn("year", year('order date (DateOrders)'))
df = df.withColumn("month", month('order date (DateOrders)'))

StatementMeta(, dab422a6-56ab-4b3e-9238-931bcf23aaf2, 9, Finished, Available)

In [8]:
path = 'abfss://4155d96d-1328-43ad-982f-7aecb1ee2008@onelake.dfs.fabric.microsoft.com/8dfa5c43-7d50-4754-9b90-3c237db830de/Files'
df.write.partitionBy('year', 'month') \
    .mode('overwrite') \
    .format('csv') \
    .option('header', 'true') \
    .save(path + '/dataset/')

StatementMeta(, dab422a6-56ab-4b3e-9238-931bcf23aaf2, 10, Finished, Available)